# Worksheet 07

Name:  Snigdha Reddy Pulim
UID: U13354202

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [73]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.415627404297768, 6.280586750606992, 6.171577218996841, 5.12023282204205, 6.552524281428266, 3.6194667532487155, 3.6622134806833824, 5.7607090167330695, 5.901381812186325, 3.516122549036929]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [74]:
c2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(c2)

[10.035992935377738, 6.04621102154703, 7.008262836271103, 8.41041753007394, 7.775790968323862, 8.349347829865021, 6.66830122769706, 7.469168927858149, 8.239251673390308, 7.604327590586914]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [75]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[3.516122549036929, 7.604327590586914, 8.239251673390308, 7.469168927858149, 6.66830122769706, 8.349347829865021, 7.775790968323862, 5.901381812186325, 8.41041753007394, 7.008262836271103]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

the parameters of this GMM are 

-> mean, 

-> standard deviation and 

-> the probabilities of the clusters 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [76]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.604327590586914, 8.239251673390308, 7.469168927858149, 6.66830122769706, 8.349347829865021, 7.775790968323862, 8.41041753007394, 7.008262836271103]
[3.516122549036929, 5.901381812186325]
P(C_1) = 0.8,  P(C_2) = 0.2
mean_1 = 7.690608573008294,  mean_2 = 4.7087521806116275
var_1 = 0.3534394465446393,  var_2 = 1.42236543811


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


-> The mean values resulting from clustering are similar to the initial mean values 

-> The probabilities of the clusters are incorrect

    -- this could be because of the random allocation in 1b 

    -- or because the data is clustered wrongly

-> The variance values of the clusters are also incorrect

    -- probably because only parts of the data from the clusters are in the data

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [77]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( (pdf_i[0]*prob_c[0])/prob_x )
    prob_c1_x.append( (pdf_i[1]*prob_c[1])/prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  3.516122549036929
probability of observing that point if it came from cluster 0 =  5.760111648945402e-31
probability of observing that point if it came from cluster 1 =  0.19734793864855762
point =  7.604327590586914
probability of observing that point if it came from cluster 0 =  1.0956061123715897
probability of observing that point if it came from cluster 1 =  0.03531697149938029
point =  8.239251673390308
probability of observing that point if it came from cluster 0 =  0.3383383361376334
probability of observing that point if it came from cluster 1 =  0.01288423619748601
point =  7.469168927858149
probability of observing that point if it came from cluster 0 =  0.9275913663207545
probability of observing that point if it came from cluster 1 =  0.042661350033794516
point =  6.66830122769706
probability of observing that point if it came from cluster 0 =  0.017213857774100328
probability of observing that point if it came from cluster 1 =  0.10858239157494308
point =  8.3493

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [78]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[1] * x[0] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7225705167654058,  P(C_2) = 0.27742948323459415
mean_1 = 7.783224931927657,  mean_2 = 5.299755513444516
var_1 = 0.2854968141264747,  var_2 = 1.9723646615910932


-> These is no significant improvement.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [79]:
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

3.516122549036929
Probability of coming from C_1 = 8.398178585752477e-48
Probability of coming from C_2 = 1.0

7.604327590586914
Probability of coming from C_1 = 0.966955473205365
Probability of coming from C_2 = 0.03304452679463493

8.239251673390308
Probability of coming from C_1 = 0.9384780522571837
Probability of coming from C_2 = 0.061521947742816246

7.469168927858149
Probability of coming from C_1 = 0.9473431215196518
Probability of coming from C_2 = 0.05265687848034821

6.66830122769706
Probability of coming from C_1 = 0.011045909359274537
Probability of coming from C_2 = 0.9889540906407254

8.349347829865021
Probability of coming from C_1 = 0.8927637347178986
Probability of coming from C_2 = 0.1072362652821013

7.775790968323862
Probability of coming from C_1 = 0.9753407807101162
Probability of coming from C_2 = 0.024659219289883783

5.901381812186325
Probability of coming from C_1 = 6.931438201590821e-09
Probability of coming from C_2 = 0.9999999930685618

8.41041753007394
Pr

-> The probability of the given data coming from a cluster have changed

-> The probability for C_2 has increased and for C_1 has decreated

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [81]:
cluster = []

for i in range(len(data)):
    if prob_c0_x[i] > prob_c1_x[i]:
        cluster.append(0)
    else:
        cluster.append(1)

print(cluster)

[1, 0, 0, 0, 1, 0, 0, 1, 0, 1]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

disagree - 1
agree - 0

| Pair | Disagreement |
|------|--------------|
| A  B |      1       |
| A  C |      0       |
| A  D |      0       |
| A  E |      0       |
| B  C |      1       |
| B  D |      0       |
| B  E |      0       |
| C  D |      0       |
| C  E |      0       |
| D  E |      1       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

n(n-1)/2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

1) A, B, C will be in disagreement with any D-I(6) = 3 * 6 = 18

2) D, E will be in disagreement with any F-I (4) = 2 * 4 = 8

maximum number of disagreements = 18 + 8 = 26

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

for cluster 1 = (2 x 1)/2 = 1
for cluster 2 = (4 x 3)/2 = 6

agreements = 3 + 1 + 6 = 10

total unique pairs = (9 x 8)/2 = 36

disagreements = total - agreements = 36 - 10 = 26

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.